In [ ]:
import pandas as pd
import numpy as np 
import os

print('Import complete')


In [ ]:
# create an empty data frame
epl_df = pd.DataFrame()
epl_df.shape

# loop through the csv files in the data-csv folder
import_dir  = 'c:\premier League\data-csv'

for entry in os.scandir(import_dir):
    if (entry.path.endswith(".csv")) and (entry.name.startswith("Season-")) and entry.is_file():   
        # print('File Name : ', entry.name)
        # create a DF from the csv file        
        df = pd.read_csv(entry, index_col=False)
        # fill any NaN values with zeroes
        # df = df.fillna(0)                               
        #append the DF to the epl_df - index all rows 1,2,3.....etc
        epl_df = pd.concat([epl_df, df], ignore_index=True)

Print('DataFrame Completed')
        

In [ ]:
epl_df.shape
# clean the data up
# remove blank rows
# epl_df.dropna(inplace = True)
# replace NaN values with blank
# epl_df.fillna('', inplace = True)
# epl_df

In [ ]:
epl_df

In [ ]:
# write the epl_df dataframe to as CSV file
# epl_df.to_csv(r'data-csv\all-seasons.csv', index = False)
# write all rows for the columns : Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR
try:
    epl_df.loc[ : , 'Date' : 'HTR' ].to_csv(r'data-csv\all-seasons.csv', index = False)
    print('File Created')
except:
    print('Something went wrong - check if file is open in Excel')

In [ ]:
# write the dataframe to as CSV file
# epl_df.to_csv(r'data-csv\all-seasons.csv', index = False)

#write only the columns : Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR
try:
    epl_df.loc[ : , 'Date' : 'HTR' ].to_csv(r'data-csv\all-seasons.csv', index = False)
    print('All Seasons data CSV created')
except:
    print('Error creating historical data CSV')


In [ ]:
historical_data = pd.read_csv(r'c:\premier league\data-csv\all-seasons.csv')
historical_data

In [ ]:
# current season data file is Season-20-21.csv

# create an empty dictionary 
league_table = dict()

# open the csv file for the current season
current_season = open(r'data-csv\Season-20-21.csv', 'rt', encoding='UTF8')

# parse each match in the CSV file matches
for match in current_season:
    
    #trim match
    match = match.rstrip()
    
    # split the match into parts, comma as separator
    match_parts = match.split(',')
    
    # skip the header line, which has "Div" as first column
    if match_parts[0] == "Div" : continue

    # should include a check at some point to see if the Time column is included in the data file
    
    ht  = match_parts[2]
    at  = match_parts[3]
    res = match_parts[6]

    # this creates a new Key or retrieves an existing Key
    # if a new Key then the Value is 0
    # if an existing Key, the Value is incremented by the points for W/L/D
    if res == 'H':
        league_table[ht] = league_table.get(ht, 0) + 3
    elif res == 'A':
        league_table[at] = league_table.get(at, 0) + 3
    elif res == 'D':
        league_table[ht] = league_table.get(ht, 0) + 1
        league_table[at] = league_table.get(at, 0) + 1

print(league_table)

# close the file handle
current_season.close()


In [ ]:
# create list of positions
position = [*range(1, len(league_table)+1, 1)]
# print(position)

# create a DataFrame to display the current league standings
premiership = pd.DataFrame.from_dict(league_table, orient='index', columns=['Points'])

# make team a column rather than an index
premiership.reset_index(inplace=True)
premiership = premiership.rename(columns = {'index':'Team'})

# sort highest to lowest
premiership.sort_values(by=['Points'], ascending=False, inplace=True)

# add the columns to indicate league position
premiership['Position'] = position

# Make the Position column the index
premiership.set_index('Position', inplace=True)

# print the league table
premiership


In [ ]:
TeamA = 'Arsenal'
TeamB = 'Leeds'

# can use SUM as there is only a single value returned so its the sum of itself
TeamAPoints = premiership.loc[premiership.Team == TeamA, 'Points'].sum()
TeamBPoints = premiership.loc[premiership.Team == TeamB, 'Points'].sum()

# get the index for the row for each team
TeamAPos = premiership[premiership['Team'] == TeamA].index[0]
TeamBPos = premiership[premiership['Team'] == TeamB].index[0]

# calculate the number of games played between the two teams
played = epl_df.loc[((epl_df["HomeTeam"].isin([TeamA, TeamB])) & (epl_df["AwayTeam"].isin([TeamA, TeamB]))), 'Date' : 'HTR' ]

# Number of games played
games_played = played.FTR.count()

# How many draws
TeamADraws    = played.loc[ ( (played.FTR == 'D') ), 'FTR' ].count()
# how many home/away wins
TeamAHomeWins = played.loc[ ( (played.FTR == 'H') & (played.HomeTeam == TeamA) ), 'FTR' ].count()
TeamAAwayWins = played.loc[ ( (played.FTR == 'A') & (played.AwayTeam == TeamA) ), 'FTR' ].count()
# how many home/away wins
TeamBHomeWins = played.loc[ ( (played.FTR == 'H') & (played.HomeTeam == TeamB) ), 'FTR' ].count()
TeamBAwayWins = played.loc[ ( (played.FTR == 'A') & (played.AwayTeam == TeamB) ), 'FTR' ].count()

# print('Played : %d  Draws : %d'%(games_played, TeamADraws))
# print('%s \t Home : %d \t Away : %d'%(TeamA, TeamAHomeWins, TeamAAwayWins))
# print('%s \t\t Home : %d \t Away : %d'%(TeamB, TeamBHomeWins, TeamBAwayWins))

print('#===================================================================================================')
print('Team \t\t Position \t Points \t Played \t Home Wins \t Away Wins \t Draws')
print('#===================================================================================================')
print('%s \t %d \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d'%(TeamA, TeamAPos, TeamAPoints, games_played, TeamAHomeWins, TeamAAwayWins, TeamADraws))
print('%s \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d'%(TeamB, TeamBPos, TeamBPoints, games_played, TeamBHomeWins, TeamBAwayWins, TeamADraws))
print('#===================================================================================================')

played


In [ ]:
from itertools import groupby

#what is the longest winning sequence TeamA has ever had
winning_sequence = (( (historical_data.loc[ ((historical_data.HomeTeam == TeamA) & (historical_data.FTR == 'H') ) | ((historical_data.AwayTeam == TeamA) & (historical_data.FTR == 'A') ), 'FTR' ]) ))

print(winning_sequence.head(30))

substrings = [list(repeated_chars) for _, repeated_chars in groupby(winning_sequence)]

# print(substrings)

longest = 0
winning_run = None

for substr in substrings:
    if (len(substr)) > longest:
        longest = len(substr)
        winning_run = substr

print('The longest winning run was %d matches, %s'%(longest, winning_run))

print(substrings)


In [125]:

#what is the longest winning sequence TeamA has had againt TeamB
winning_sequence = (( (played.loc[ ((played.HomeTeam == TeamA) & (played.FTR == 'H') ) | ((played.AwayTeam == TeamA) & (played.FTR == 'A') ), 'FTR' ]) ))

print(winning_sequence.head(30))

substrings = [list(repeated_chars) for _, repeated_chars in groupby(winning_sequence)]

# print(substrings)

longest = 0
winning_run = None

for substr in substrings:
    if (len(substr)) > longest:
        longest = len(substr)
        winning_run = substr

print('The longest winning run was %d matches, %s'%(longest, winning_run))

print(substrings)


356      H
833      A
1241     A
1467     H
7832     H
7911     H
8884     A
9123     H
9280     H
9769     H
10119    H
10507    H
10650    A
Name: FTR, dtype: object
The longest winning run was 5 matches, ['H', 'H', 'H', 'H', 'H']
[['H'], ['A', 'A'], ['H', 'H', 'H'], ['A'], ['H', 'H', 'H', 'H', 'H'], ['A']]
